In [1]:
import pandas as pd
import re

Create dataset for the corresponding year

In [2]:
# dataset = pd.read_csv('zno_preprocessed.csv')
# dataset_grad = dataset[(dataset.regtypename == 'Випускник української школи поточного року') & (dataset.year == 2018)]
# dataset_schools = dataset_grad[['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']]
# dataset_schools_unique=dataset_schools.drop_duplicates()
# dataset_schools_unique.to_csv('schools2018.csv', index=False)

download dataset from EIE

In [3]:
eie_schools = pd.read_csv('schools2018.csv')

In [4]:
eie_schools.shape

(9773, 6)

In [5]:
eie_schools.eoregname

0           Закарпатська область
1               Київська область
2             Харківська область
3             Рівненська область
4             Запорізька область
                  ...           
9768        Чернігівська область
9769    Дніпропетровська область
9770          Харківська область
9771          Рівненська область
9772    Дніпропетровська область
Name: eoregname, Length: 9773, dtype: object

Create new column with region name and territory name in the format that matched with finance dataset

In [6]:
eie_schools['region'] = eie_schools.eoregname.str.split().str[0]
eie_schools['region'] = eie_schools['region'].str.replace('м.Київ', 'місто Київ')
eie_schools['eoareaname'] = eie_schools['eoareaname'].str.replace('м. ', 'м.')
eie_schools['tername'] = eie_schools.eoareaname.str.split().str[0]
eie_schools['tername'] = eie_schools.tername.str.rstrip('.')


pattern = r'((I-(III|II))|(І-(ІІІ|ІІ)))'
eie_schools['level'] = eie_schools.eoname.str.extract(pattern)[0]

dct = {'i':'і', 'o':'о', 'y':'у', 'p':'р', 'c':'с', 'ґ':'г', "’": "'"}

for symb in dct:
    eie_schools.tername = eie_schools.tername.str.replace(symb, dct[symb])

eie_schools['tername']= eie_schools['tername'].str.replace('м.Лозова', 'Лозівський')
eie_schools['tername']= eie_schools['tername'].str.replace('м.Біляївка', 'Біляївський')
eie_schools.loc[eie_schools['tername'].str.startswith('Подільський'), 'tername'] = 'м.Подільськ'
eie_schools['tername']= eie_schools['tername'].str.replace('м.Лиман', 'Лиманський')
eie_schools['tername']= eie_schools['tername'].str.replace('м.Балта', 'Балтський')
eie_schools['tername']= eie_schools['tername'].str.replace('м.Канів', 'Канівський')
eie_schools['tername']= eie_schools['tername'].str.replace('Кіровоградський', 'Кропивницький')
eie_schools.loc[eie_schools['tername'].str.startswith('Переяслав-Хмельницький'), 'tername'] = 'м.' + eie_schools['tername']




eie_schools

,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,region,tername,level
0,Косоньська загальноосвітня школа І-ІІІ ступені...,середня загальноосвітня школа,Закарпатська область,Берегівський район,с.Косонь,"Відділ освіти, молоді та спорту Берегівської р...",Закарпатська,Берегівський,І-ІІІ
1,Броварська спеціалізована школа І-ІІІ ступенів...,спеціалізована школа,Київська область,м.Бровари,м.Бровари,Управління освіти і науки Броварської міської ...,Київська,м.Бровари,І-ІІІ
2,Харківська гімназія № 12 Харківської міської р...,гімназія,Харківська область,м.Харків. Червонозаводський район міста,Червонозаводський район міста,Управління освіти адміністрації Основ'янського...,Харківська,м.Харків,NaN
3,Млинівська загальноосвітня школа І-ІІІ ступені...,середня загальноосвітня школа,Рівненська область,Млинівський район,смт Млинів,"Управління освіти, культури, туризму, молоді т...",Рівненська,Млинівський,І-ІІІ
4,"Таврійський навчально-виховний комплекс "" Зага...",навчально-виховний комплекс,Запорізька область,Токмацький район,с.Таврія,Відділ освіти Токмацької районної державної ад...,Запорізька,Токмацький,І-ІІІ
...,...,...,...,...,...,...,...,...,...
9768,Ядутинська загальноосвітня школа I-III ступені...,середня загальноосвітня школа,Чернігівська область,Борзнянський район,с.Ядути,Відділ освіти Борзнянської районної державної ...,Чернігівська,Борзнянський,I-III
9769,"Комунальний заклад ""Софіївська вечірня школа"" ...",вечірня (змінна) школа,Дніпропетровська область,Софіївський район,смт Софіївка,Відділ освіти виконавчого комітету Софіївської...,Дніпропетровська,Софіївський,NaN
9770,Вільхівська загальноосвітня школа I-III ступен...,середня загальноосвітня школа,Харківська область,Харківський район,с.Вільхівка,Відділ освіти Харківської районної державної а...,Харківська,Харківський,I-III
9771,Смордвівська загальноосвітня школа І-ІІІ ступе...,середня загальноосвітня школа,Рівненська область,Млинівський район,с.Смордва,"Відділ освіти, молоді та спорту, культури, тур...",Рівненська,Млинівський,І-ІІІ


download finance dataset

In [7]:
finance_schools = pd.read_excel('ZZSO-2018.xlsx', header = 1)

In [8]:
finance_schools.head()

,№ Області,Область,Тип територіальної одиниці,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Питома вага непедагогічного персоналу у загальній кількості працівників, %**","Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**",Кількість учнів разом,Кількість класів разом,"Фактична наповнюваність класів, учнів на клас**","Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (план)","Видатки на 1 учня, 2018, тис. грн.**","Видатки на 1 клас, тис. грн.** 2018 рік (каса)"
0,2,Вінницька,Район,391,Погребищенський,КЗ «Ординецька ЗОШ І ступеня Погреб районної р...,I,888,0.0,0.0,...,NaN,NaN,0,0,NaN,NaN,0.000,0,NaN,NaN
1,4,Дніпропетровська,Район,1891,Магдалинівський район,Олександрівська ЗОШ І ступеня Магдалинівської ...,I,1467.4,0.0,4.0,...,1.0,NaN,0,0,NaN,NaN,237.494,0,NaN,NaN
2,4,Дніпропетровська,Район,1892,Магдалинівський район,Крамарська загальноосвітня школа І-ІІ ступенів...,I-II,524.21,0.0,2.0,...,1.0,NaN,0,0,NaN,NaN,821.143,0,NaN,NaN
3,4,Дніпропетровська,Район,1893,Магдалинівський район,Минівська ЗОШ І ступеня Заплавської сільської ...,I,881,0.0,3.0,...,1.0,NaN,0,0,NaN,NaN,343.575,0,NaN,NaN
4,4,Дніпропетровська,Район,1962,П'ятихатський район,Новотроїцька загальноосвітня школа I- II ступе...,I-II,205.7,0.0,0.0,...,NaN,NaN,0,0,NaN,NaN,927.012,0,NaN,NaN


In [9]:
finance_schools.columns

Index(['№ Області', 'Область', 'Тип територіальної одиниці', '№№',
       'Назва тер. Одиниці', 'Найменування закладу',
       'Ступінь закладу ( I, I-II, I-III)',
       'Загальна площа приміщень школи ( без господарських будівель) (кв.м.)',
       'Кількість штатних працівників педагогічного персоналу  разом',
       'Кількість штатних працівників непедагогічного персоналу  разом',
       'Кількість штатних працівників педагогічного персоналу  пенсійного віку',
       'Питома вага непедагогічного персоналу у загальній кількості працівників, %**',
       'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**',
       'Кількість учнів разом', 'Кількість класів разом',
       'Фактична наповнюваність класів, учнів на клас**',
       'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**',
       'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)',
       'Видатки на функціонув

choose columns that give information about schools

In [10]:
finance_schools = finance_schools[['Область', 'Назва тер. Одиниці', 'Найменування закладу', 'Ступінь закладу ( I, I-II, I-III)']]

Rename columns using a dictionary 

In [11]:
new_columns = {'Область': 'region', 'Назва тер. Одиниці': 'teritname', 'Найменування закладу': 'schoolname', 'Ступінь закладу ( I, I-II, I-III)': 'level of school'}
finance_schools = finance_schools.rename(columns=new_columns)

In [12]:
finance_schools.teritname = finance_schools.teritname.str.replace('м. ', 'м.')
finance_schools.teritname = finance_schools.teritname.str.replace('с. ', 'с.')
finance_schools.teritname = finance_schools.teritname.str.replace('смт. ', 'смт.')
finance_schools.teritname = finance_schools.teritname.str.replace('смт ', 'смт.')
finance_schools['region'] = finance_schools['region'].str.replace('м.Київ', 'місто Київ')
finance_schools['tername'] = finance_schools.teritname
finance_schools['tername'] = finance_schools['tername'].str.replace(r'\d+', '', regex=True)
finance_schools['tername'] = finance_schools['tername'].apply(lambda x: x[x.find('(')+1:x.find(')')] if '(' in x else x)


finance_schools['tername']= finance_schools['tername'].str.replace('ОТГ ', '')
finance_schools['tername']= finance_schools['tername'].str.replace('ОТГ.', '')

finance_schools['tername'] = finance_schools.tername.str.split().str[0]

finance_schools['tername'] = finance_schools['tername'].str.rstrip('-')

finance_schools['tername'] = finance_schools['tername'].apply(lambda x: x[:-1] + 'ий' if (x.endswith('а') & ~(x.startswith('м.'))) else x)
finance_schools['tername'] = finance_schools['tername'].apply(lambda x: x[:-3] + 'ий' if x.endswith('ого') else x)
finance_schools['tername'] = finance_schools['tername'].str.replace('Київ', 'м.Київ')
finance_schools['tername'] = finance_schools['tername'].str.replace('Кам-Подільський', "Кам'янець-Подільський")
finance_schools['tername'] = finance_schools['tername'].str.replace('м.Вугледару', 'м.Вугледар')

dct = {'i':'і', 'o':'о', 'y':'у', 'p':'р', 'c':'с', 'ґ':'г', "’": "'"}

for symb in dct:
    finance_schools['tername'] = finance_schools['tername'].str.replace(symb, dct[symb])


finance_schools['tername']= finance_schools['tername'].str.replace('м.Староконстантинiв', 'м.Старокостянтинiв')

finance_schools['tername']= finance_schools['tername'].str.replace('Вишгороський', 'Вишгородський')
finance_schools['tername']= finance_schools['tername'].str.replace('м.Берегово', 'м.Берегове')
finance_schools['tername']= finance_schools['tername'].str.replace('Іваничівськиій', 'Іваничівський')
finance_schools['tername']= finance_schools['tername'].str.replace('Старовиживський', 'Старовижівський')
finance_schools['tername']= finance_schools['tername'].str.replace(r'Вінн$', 'Вінницький')
finance_schools['tername']= finance_schools['tername'].str.replace(r'Гайс$', 'Гайсинський')
finance_schools['tername']= finance_schools['tername'].str.replace(r'Калин$', 'Калинівський')
finance_schools['tername']= finance_schools['tername'].str.replace(r'Козятин$', 'Козятинський')
finance_schools['tername']= finance_schools['tername'].str.replace(r'Липов$', 'Липовецький')
finance_schools['tername']= finance_schools['tername'].str.replace(r'Тиврівс$', 'Тиврівський')
finance_schools['tername']= finance_schools['tername'].str.replace(r'Чернів$', 'Чернівецький')
finance_schools['tername']= finance_schools['tername'].str.replace('Радомишль', 'Радомишльський')
finance_schools['tername']= finance_schools['tername'].str.replace('Ст.Самбірський', 'Старосамбірський')
finance_schools['tername']= finance_schools['tername'].str.replace('Кам.-Бузький', "Кам'янка-Бузький")
finance_schools['tername']= finance_schools['tername'].str.replace('м.Староконстантинів', 'м.Старокостянтинів')
finance_schools.loc[finance_schools['tername'].str.startswith('Переяслав-Хмельницький'), 'tername'] = 'м.' + finance_schools['tername']
finance_schools['tername']= finance_schools['tername'].str.replace('"', "'", regex = True)

In [13]:
finance_schools.head(20)

,region,teritname,schoolname,level of school,tername
0,Вінницька,Погребищенський,КЗ «Ординецька ЗОШ І ступеня Погреб районної р...,I,Погребищенський
1,Дніпропетровська,Магдалинівський район,Олександрівська ЗОШ І ступеня Магдалинівської ...,I,Магдалинівський
2,Дніпропетровська,Магдалинівський район,Крамарська загальноосвітня школа І-ІІ ступенів...,I-II,Магдалинівський
3,Дніпропетровська,Магдалинівський район,Минівська ЗОШ І ступеня Заплавської сільської ...,I,Магдалинівський
4,Дніпропетровська,П'ятихатський район,Новотроїцька загальноосвітня школа I- II ступе...,I-II,П'ятихатський
5,Дніпропетровська,П'ятихатський район,"Троїцький навчально- виховний комплекс ""загаль...",I-III,П'ятихатський
6,Дніпропетровська,П'ятихатський район,Чистопільська загальноосвітня школа І-ІІ ступе...,I-II,П'ятихатський
7,Дніпропетровська,Царичанська ОТГ (Царичанський район),Михайлівська ЗОШ І-ступеня,I,Царичанський
8,Дніпропетровська,Царичанська ОТГ (Царичанський район),"Юр""ївська загальноосвітня школа І ступеня",I,Царичанський
9,Дніпропетровська,Вакулівська ОТГ (Софіївський район),"Комунальний заклад ""Нововасилівська загальноос...",I,Софіївський


In [14]:
finance_schools.shape

(14776, 5)

In [15]:
eie_schools.shape

(9773, 9)

In [16]:
set(eie_schools.region) == set(finance_schools.region)

True

In [17]:
set(eie_schools.tername) - set(finance_schools.tername)

set()

In [18]:
df = pd.DataFrame(columns=['eoname', 'schoolname'])
df

,eoname,schoolname


check for duplicates by the name of school

In [19]:
for region in sorted(eie_schools.region.unique()):
    region_schools = eie_schools[eie_schools.region == region]
    region_finance = finance_schools[finance_schools.region == region]
    if len(region_schools[region_schools.eoname.duplicated(keep=False)]) and len(region_finance[region_finance.schoolname.duplicated(keep=False)]):
        print('eie')
        display(region_schools[region_schools.eoname.duplicated(keep=False)].sort_values('eoname'))
        print('finance')
        display(region_finance[region_finance.schoolname.duplicated(keep=False)].sort_values('schoolname'))


eie


,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,region,tername,level
8768,Яланецька загальноосвітня школа I-III ступенів,середня загальноосвітня школа,Вінницька область,Томашпільський район,с.Яланець,Відділ освіти Томашпільської районної державно...,Вінницька,Томашпільський,I-III
9497,Яланецька загальноосвітня школа I-III ступенів,середня загальноосвітня школа,Вінницька область,Бершадський район,с.Яланець,"Відділ освіти, фізичної культури та спорту Бер...",Вінницька,Бершадський,I-III


finance


,region,teritname,schoolname,level of school,tername
274,Вінницька,Жмеринський,Браїлівська,I-III,Жмеринський
281,Вінницька,Жмеринський,Браїлівська,I-II,Жмеринський
370,Вінницька,Крижопільський,СЗШ с Соколівка,I-III,Крижопільський
371,Вінницька,Крижопільський,СЗШ с Соколівка,I-II,Крижопільський


eie


,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,region,tername,level
6334,Березівська загальноосвітня школа I-III ступенів,середня загальноосвітня школа,Житомирська область,Любарський район,с.Березівка,Відділ освіти Любарської районної державної ад...,Житомирська,Любарський,I-III
8894,Березівська загальноосвітня школа I-III ступенів,середня загальноосвітня школа,Житомирська область,Житомирський район,с.Березівка,Відділ освіти Житомирської районної державної ...,Житомирська,Житомирський,I-III
1649,Варварівська загальноосвітня школа I-III ступенів,середня загальноосвітня школа,Житомирська область,Ємільчинський район,с.Варварівка,Відділ освіти Ємільчинської районної державної...,Житомирська,Ємільчинський,I-III
8244,Варварівська загальноосвітня школа I-III ступенів,середня загальноосвітня школа,Житомирська область,Олевський район,с.Варварівка,"Відділ освіти, молоді та спорту Олевської місь...",Житомирська,Олевський,I-III
6557,Вільшанська загальноосвітня школа I-III ступенів,середня загальноосвітня школа,Житомирська область,Чуднівський район,с.Вільшанка,Відділ освіти Чуднівської районної державної а...,Житомирська,Чуднівський,I-III
9364,Вільшанська загальноосвітня школа I-III ступенів,середня загальноосвітня школа,Житомирська область,Романівський район,с.Вільха,Відділ освіти Романівської районної державної ...,Житомирська,Романівський,I-III


finance


,region,teritname,schoolname,level of school,tername
2998,Житомирська,Житомирський район,Іванівська ЗОШ,I-III,Житомирський
3071,Житомирська,Малинський район,Іванівська ЗОШ,I-II,Малинський
2992,Житомирська,Житомирський район,Барашівська ЗОШ,I-III,Житомирський
3187,Житомирська,Барашівська ОТГ,Барашівська ЗОШ,I-III,Барашівський
3295,Житомирська,Любарська ОТГ,Березівська ЗОШ,I-III,Любарський
2993,Житомирська,Житомирський район,Березівська ЗОШ,I-III,Житомирський
3199,Житомирська,Брониківська ОТГ( ОТГ Романівка),Варварівська ЗОШ,I-II,Романівкий
2983,Житомирська,Ємільчинський район,Варварівська ЗОШ,I-III,Ємільчинський
3345,Житомирська,Олевська ОТГ,Варварівська ЗОШ,I-III,Олевський
3173,Житомирська,Андрушківська ОТГ,Василівська ЗОШ,I-II,Андрушківський


resolving duplicates

In [20]:
eie_schools.loc[6334, 'eoname'] = eie_schools.loc[6334, 'eoname']+ 'Любарського'
finance_schools.loc[3295, 'schoolname'] = finance_schools.loc[3295, 'schoolname']+ 'Любарського'

In [21]:
eie_schools.loc[1649, 'eoname'] = eie_schools.loc[1649, 'eoname']+ 'Ємил'
finance_schools.loc[2983, 'schoolname'] = finance_schools.loc[2983, 'schoolname']+ 'Ємил'

In [22]:
eie_schools.loc[9364, 'eoname'] = eie_schools.loc[9364, 'eoname']+ 'Роман'
finance_schools.loc[3117, 'schoolname'] = finance_schools.loc[3117, 'schoolname']+ 'Роман'

In [23]:
for region in sorted(eie_schools.region.unique()):
    region_schools = eie_schools[eie_schools.region == region]
    region_finance = finance_schools[finance_schools.region == region]
    if len(region_schools[region_schools.eoname.duplicated(keep=False)]) and len(region_finance[region_finance.schoolname.duplicated(keep=False)]):
        print('eie')
        display(region_schools[region_schools.eoname.duplicated(keep=False)].sort_values('eoname'))
        print('finance')
        display(region_finance[region_finance.schoolname.duplicated(keep=False)].sort_values('schoolname'))


eie


,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,region,tername,level
8768,Яланецька загальноосвітня школа I-III ступенів,середня загальноосвітня школа,Вінницька область,Томашпільський район,с.Яланець,Відділ освіти Томашпільської районної державно...,Вінницька,Томашпільський,I-III
9497,Яланецька загальноосвітня школа I-III ступенів,середня загальноосвітня школа,Вінницька область,Бершадський район,с.Яланець,"Відділ освіти, фізичної культури та спорту Бер...",Вінницька,Бершадський,I-III


finance


,region,teritname,schoolname,level of school,tername
274,Вінницька,Жмеринський,Браїлівська,I-III,Жмеринський
281,Вінницька,Жмеринський,Браїлівська,I-II,Жмеринський
370,Вінницька,Крижопільський,СЗШ с Соколівка,I-III,Крижопільський
371,Вінницька,Крижопільський,СЗШ с Соколівка,I-II,Крижопільський


In [24]:
eie_schools_temp = eie_schools
finance_schools_temp = finance_schools

In [25]:
for region in sorted(eie_schools.region.unique()):
    for eoname in eie_schools[eie_schools.region == region].eoname:
        if eoname in finance_schools[finance_schools.region == region].schoolname.unique():
            df = pd.concat([df, pd.DataFrame([{'eoname': eoname, 'schoolname': eoname}])], ignore_index=True)
            eie_schools_temp = eie_schools_temp[~((eie_schools_temp.region == region) & (eie_schools_temp.eoname == eoname))]
            finance_schools_temp = finance_schools_temp[~((finance_schools_temp.region == region) & (finance_schools_temp.schoolname == eoname))]

df

,eoname,schoolname
0,Нововолинська гімназія Нововолинської міської ...,Нововолинська гімназія Нововолинської міської ...
1,Шаргородська районна гімназія,Шаргородська районна гімназія
2,"Комунальний заклад ""Вінницький технічний ліцей""","Комунальний заклад ""Вінницький технічний ліцей"""
3,Комунальний заклад загальної середньої освіти ...,Комунальний заклад загальної середньої освіти ...
4,Марганецька загальноосвітня школа І-ІІІ ступен...,Марганецька загальноосвітня школа І-ІІІ ступен...
...,...,...
647,Спеціалізована школа І-ІІІ ступенів № 202 з по...,Спеціалізована школа І-ІІІ ступенів № 202 з по...
648,Школа І-ІІІ ступенів № 213 Деснянського району...,Школа І-ІІІ ступенів № 213 Деснянського району...
649,Кловський ліцей № 77,Кловський ліцей № 77
650,Школа І-ІІІ ступенів № 190 Деснянського району...,Школа І-ІІІ ступенів № 190 Деснянського району...


control the number of rows added and removed

In [26]:
print(eie_schools.shape[0]-eie_schools_temp.shape[0])
print(finance_schools.shape[0] - finance_schools_temp.shape[0])
print(df.shape[0])

652
652
652


Creating id from school's name

In [27]:
eie_schools_temp['id'] = eie_schools_temp['eoname']

abbr = {'КЗ ':'Комунальний заклад ', 'ОНЗ ':'Опорний навчальний заклад ', 'ОЗЗСО ':'Опорний заклад загальної середньої освіти ',
        'ЗЗСО ':'Заклад загальної середньої освіти ', 'ООЗ ':'Опорний освітній заклад ', 'ЗСО ':'Загальної середньої освіти ', 
        'НВК ':'Навчально-виховний комплекс ', 'ЗОШ ':'Загальнооосвітня школа ', 'СЗШ ':'Середня загальноосвітня школа ',
        'ДНЗ ':'Дошкільний навчальний заклад '}

for ab in abbr:
    eie_schools_temp['id'] = eie_schools_temp['id'].str.replace(ab, abbr[ab])

eie_schools_temp['id'] = eie_schools_temp['id'].str.replace('[\s]', '', regex = True)
eie_schools_temp.loc[eie_schools_temp['id'].str.endswith('обл'), 'id'] = eie_schools_temp.loc[eie_schools_temp['id'].str.endswith('обл'), 'id'] + '.'
eie_schools_temp['id'] = eie_schools_temp['id'].str.replace('І', 'I')
eie_schools_temp['id'] = eie_schools_temp['id'].str.replace('I-III', '3')
eie_schools_temp['id'] = eie_schools_temp['id'].str.replace('I-II', '2')
eie_schools_temp['id'] = eie_schools_temp['id'].str.lower()

dct = {'с.':'села', 'ст.':'ступенів', 'ім.':'імені', 'iм.':'імені', 'обл.': 'області', 
       '||||':'3', '|||':'2',
       'i':'', 'і':'', 'ї':'', 'o':'о', 'y':'у', 'p':'р', 'c':'с', 'ґ':'', 
       "'":'', "’":'', ",":'', ".":'', "«":'', "»":'', "–":'', "-":'', "*":"",
       "_x000d_":'', '_x000D_':'', "_":'', "(":'', ")":'', "[":'', "]":'', "}":'', "{":'', 
       "нвк":'навчальновиховнийкомплекс', 'знз': 'загальноосвтнйнавчальнийзаклад', 'зош': 'загальноосвтняшкола', 
       'сзш':'середнязагальноосвтняшкола', 'нн':'н', 'днз': 'дошкльнийнавчальнийзаклад'}

for symb in dct:
    eie_schools_temp['id'] = eie_schools_temp['id'].str.replace(symb, dct[symb])

eie_schools_temp['id'] = eie_schools_temp['id'].str.replace('"', '', regex = True)


In [28]:
eie_schools_temp.head(10)

,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,region,tername,level,id
1,Броварська спеціалізована школа І-ІІІ ступенів...,спеціалізована школа,Київська область,м.Бровари,м.Бровари,Управління освіти і науки Броварської міської ...,Київська,м.Бровари,І-ІІІ,броварськаспецалзованашкола3ступенв№7броварськ...
3,Млинівська загальноосвітня школа І-ІІІ ступені...,середня загальноосвітня школа,Рівненська область,Млинівський район,смт Млинів,"Управління освіти, культури, туризму, молоді т...",Рівненська,Млинівський,І-ІІІ,млинвськазагальноосвтняшкола3ступенв№1млинвськ...
4,"Таврійський навчально-виховний комплекс "" Зага...",навчально-виховний комплекс,Запорізька область,Токмацький район,с.Таврія,Відділ освіти Токмацької районної державної ад...,Запорізька,Токмацький,І-ІІІ,таврйськийнавчальновиховнийкомплексзагальноосв...
6,Новоград-Волинська загальноосвітня школа І-ІІІ...,середня загальноосвітня школа,Житомирська область,м.Новоград-Волинський,м.Новоград-Волинський,Управління освіти і науки Новоград-Волинської ...,Житомирська,м.Новоград-Волинський,І-ІІІ,новоградволинськазагальноосвтняшкола3ступенв№2...
8,"УМАНСЬКИЙ НАВЧАЛЬНО - ВИХОВНИЙ КОМПЛЕКС ""ЗАГАЛ...",середня загальноосвітня школа,Черкаська область,м.Умань,м.Умань,Відділ освіти Уманської міської ради,Черкаська,м.Умань,І-ІІІ,уманськийнавчальновиховнийкомплексзагальноосвт...
9,Перегінська загальноосвітня школа І-ІІІ ступен...,середня загальноосвітня школа,Івано-Франківська область,Рожнятівський район,смт Перегінське,"Управління освіти, молоді та спорту Рожнятівсь...",Івано-Франківська,Рожнятівський,І-ІІІ,перегнськазагальноосвтняшкола3ступенв№2рожнятв...
10,Спеціалізована школа №304 Святошинського район...,спеціалізована школа,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,"Управління освіти, молоді та спорту Святошинсь...",місто Київ,м.Київ,NaN,спецалзованашкола№304святошинськогорайонумкиєва
11,Нагірнянська загальноосвітня школа І-ІІІ ступе...,середня загальноосвітня школа,Черкаська область,Жашківський район,с.Нагірна,Відділ освіти Жашківської районної державної а...,Черкаська,Жашківський,І-ІІІ,нагрнянськазагальноосвтняшкола3ступенвжашквськ...
12,Навчально-виховний комплекс №3 «загальноосвітн...,навчально-виховний комплекс,Вінницька область,м.Могилів-Подільський,м.Могилів-Подільський,Управління освіти Могилів-Подільської міської ...,Вінницька,м.Могилів-Подільський,NaN,навчальновиховнийкомплекс№3загальноосвтнйнавча...
13,"Чкаловський навчально-виховний комплекс ""Загал...",навчально-виховний комплекс,Запорізька область,Приазовський район,с.Чкалове,"Відділ освіти, молоді та спорту Приазовської р...",Запорізька,Приазовський,І-ІІІ,чкаловськийнавчальновиховнийкомплексзагальноос...


In [29]:
finance_schools_temp['id'] = finance_schools_temp.schoolname
abbr = {'КЗ ':'Комунальний заклад ', 'ОНЗ ':'Опорний навчальний заклад ', 'ОЗЗСО ':'Опорний заклад загальної середньої освіти ',
        'ЗЗСО ':'Заклад загальної середньої освіти ', 'ООЗ ':'Опорний освітній заклад ', 'ЗСО ':'Загальної середньої освіти ', 
        'НВК ':'Навчально-виховний комплекс ', 'ЗОШ ':'Загальнооосвітня школа ', 'СЗШ ':'Середня загальноосвітня школа ',
        'ДНЗ ':'Дошкільний навчальний заклад '}

for ab in abbr:
    finance_schools_temp['id'] = finance_schools_temp['id'].str.replace(ab, abbr[ab])

finance_schools_temp['id'] = finance_schools_temp['id'].str.replace('[\s]', '', regex = True)
finance_schools_temp.loc[finance_schools_temp['id'].str.endswith('обл'), 'id'] = finance_schools_temp.loc[finance_schools_temp['id'].str.endswith('обл'), 'id'] + '.'
finance_schools_temp.loc[finance_schools_temp['id'].str.endswith('ст'), 'id'] = finance_schools_temp.loc[finance_schools_temp['id'].str.endswith('ст'), 'id'] + '.'
finance_schools_temp['id'] = finance_schools_temp['id'].str.replace('І', 'I')
finance_schools_temp['id'] = finance_schools_temp['id'].str.replace('I-III', '3')
finance_schools_temp['id'] = finance_schools_temp['id'].str.replace('I-II', '2')
finance_schools_temp['id'] = finance_schools_temp['id'].str.lower()

dct = {'с.':'села', 'ст.':'ступенів', 'ім.':'імені', 'iм.':'імені', 'обл.': 'області', 
       '||||':'3', '|||':'2',
       'i':'', 'і':'', 'ї':'', 'o':'о', 'y':'у', 'p':'р', 'c':'с', 'ґ':'', 
       "'":'', "’":'', ",":'', ".":'', "«":'', "»":'', "–":'', "-":'', "*":"",
       "_x000d_":'', '_x000D_':'', "_":'', "(":'', ")":'', "[":'', "]":'', "}":'', "{":'', 
       "нвк":'навчальновиховнийкомплекс', 'знз': 'загальноосвтнйнавчальнийзаклад', 'зош': 'загальноосвтняшкола', 
       'сзш':'середнязагальноосвтняшкола', 'нн':'н', 'днз': 'дошкльнийнавчальнийзаклад'}

for symb in dct:
    finance_schools_temp['id'] = finance_schools_temp['id'].str.replace(symb, dct[symb])

finance_schools_temp['id'] = finance_schools_temp['id'].str.replace('"', '', regex = True)
finance_schools_temp.head()

,region,teritname,schoolname,level of school,tername,id
0,Вінницька,Погребищенський,КЗ «Ординецька ЗОШ І ступеня Погреб районної р...,I,Погребищенський,комунальнийзакладординецьказагальнооосвтняшкол...
1,Дніпропетровська,Магдалинівський район,Олександрівська ЗОШ І ступеня Магдалинівської ...,I,Магдалинівський,олександрвськазагальнооосвтняшколаступенямагда...
2,Дніпропетровська,Магдалинівський район,Крамарська загальноосвітня школа І-ІІ ступенів...,I-II,Магдалинівський,крамарськазагальноосвтняшкола2ступенвжданвсько...
3,Дніпропетровська,Магдалинівський район,Минівська ЗОШ І ступеня Заплавської сільської ...,I,Магдалинівський,минвськазагальнооосвтняшколаступенязаплавськос...
4,Дніпропетровська,П'ятихатський район,Новотроїцька загальноосвітня школа I- II ступе...,I-II,П'ятихатський,новотроцьказагальноосвтняшкола2ступенвпятихатс...


check duplicates inside regions of interest for both datasets

In [30]:
for region in sorted(eie_schools_temp.region.unique()):
    region_school = eie_schools_temp[eie_schools_temp.region == region]
    region_finance = finance_schools_temp[finance_schools_temp.region == region]
    for id in region_school.id:
        if id in region_finance.id.unique():
            if len(region_school[region_school.id == id]) > 1 or len(region_finance[region_finance.id == id]) > 1:
                print('eie')
                display(region_school[region_school.id == id])
                print('finance')
                display(region_finance[region_finance.id == id])


eie


,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,region,tername,level,id
6866,ІІ-Іванівська загальноосвітня школа I-III ступ...,середня загальноосвітня школа,Харківська область,Барвінківський район,с-ще Іванівка,Відділ освіти Барвінківської районної державно...,Харківська,Барвінківський,I-III,ванвськазагальноосвтняшкола3ступенвбарвнквсько...


finance


,region,teritname,schoolname,level of school,tername,id
11463,Харківська,Барвінківський,І-Іванівська загальноосвітня школа І-ІІІ ступе...,I-III,Барвінківський,ванвськазагальноосвтняшкола3ступенвбарвнквсько...
11464,Харківська,Барвінківський,ІІ-Іванівська загальноосвітня школа І-ІІІ ступ...,I-III,Барвінківський,ванвськазагальноосвтняшкола3ступенвбарвнквсько...


resolving duplicates

In [31]:
eie_schools_temp.loc[6866, 'id'] = 'II' + eie_schools_temp.loc[6866, 'id']
finance_schools_temp.loc[11464	, 'id'] = 'II'+finance_schools_temp.loc[11464	, 'id']

checking again

In [32]:
for region in sorted(eie_schools_temp.region.unique()):
    region_school = eie_schools_temp[eie_schools_temp.region == region]
    region_finance = finance_schools_temp[finance_schools_temp.region == region]
    for id in region_school.id:
        if id in region_finance.id.unique():
            if len(region_school[region_school.id == id]) > 1 or len(region_finance[region_finance.id == id]) > 1:
                print('data')
                display(region_school[region_school.id == id])
                print('finance')
                display(region_finance[region_finance.id == id])

In [33]:
eie_schools_temp1 = eie_schools_temp
finance_schools_temp1 = finance_schools_temp

matching by id

In [34]:
for region in sorted(eie_schools_temp.region.unique()):
    region_school = eie_schools_temp[eie_schools_temp.region == region]
    region_finance = finance_schools_temp[finance_schools_temp.region == region]
    for id in region_school.id:
        if id in region_finance.id.unique():
            df = pd.concat([df, pd.DataFrame([{'eoname': eie_schools_temp.loc[eie_schools_temp.id == id, 'eoname'].iloc[0], 'schoolname': finance_schools_temp.loc[finance_schools_temp.id == id, 'schoolname'].iloc[0]}])], ignore_index=True)
            eie_schools_temp1 = eie_schools_temp1[~((eie_schools_temp1.region == region) & (eie_schools_temp1.id == id))]
            finance_schools_temp1 = finance_schools_temp1[~((finance_schools_temp1.region == region) & (finance_schools_temp1.id == id))]

df.shape

(2885, 2)

In [35]:
print(eie_schools.shape[0]-eie_schools_temp1.shape[0])
print(finance_schools.shape[0] - finance_schools_temp1.shape[0])
print(df.shape[0])

2885
2885
2885


In [36]:
df

,eoname,schoolname
0,Нововолинська гімназія Нововолинської міської ...,Нововолинська гімназія Нововолинської міської ...
1,Шаргородська районна гімназія,Шаргородська районна гімназія
2,"Комунальний заклад ""Вінницький технічний ліцей""","Комунальний заклад ""Вінницький технічний ліцей"""
3,Комунальний заклад загальної середньої освіти ...,Комунальний заклад загальної середньої освіти ...
4,Марганецька загальноосвітня школа І-ІІІ ступен...,Марганецька загальноосвітня школа І-ІІІ ступен...
...,...,...
2880,Спеціалізована загальноосвітня школа І-ІІІ сту...,спеціалізована загальноосвітня школа І-ІІІ сту...
2881,Школа І-ІІІ ступенів №111 імені С.А.Ковпака Да...,школа І-ІІІ ступенів №111 ім. С.А.Ковпака Дарн...
2882,Спеціалізована загальноосвітня школа І-ІІІ сту...,спеціалізована загальноосвітня школа І-ІІІ сту...
2883,Школа І-ІІІ ступенів №217 Дарницького району м...,школа І-ІІІ ступенів №217 Дарницького району м...


matching by id trimming to the length of the other id

In [37]:
eie_schools_temp2 = eie_schools_temp1
finance_schools_temp2 = finance_schools_temp1

In [38]:
(eie_schools.shape[0]-eie_schools_temp2.shape[0]) == (finance_schools.shape[0]-finance_schools_temp2.shape[0])

True

In [39]:
for region in sorted(eie_schools_temp1.region.unique()):
    region_schools = eie_schools_temp1[eie_schools_temp1.region == region]
    region_finance = finance_schools_temp1[finance_schools_temp1.region == region]
    for id in region_schools.id:
        if (id in region_finance.id.str[:len(id)].unique()) and \
             re.findall(r'№(\d+)', region_schools.loc[region_schools.id == id, 'eoname'].iloc[0]) == \
                re.findall(r'№(\d+)', region_finance.loc[region_finance.id.str[:len(id)] == id, 'schoolname'].iloc[0]):
            if len(region_schools[region_schools.id == id])>1 or len(region_finance[region_finance.id.str[:len(id)] == id]) >1:
                print('eie')
                display(region_schools[region_schools.id == id])
                print('finance')
                display(region_finance[region_finance.id.str[:len(id)] == id])

eie


,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,region,tername,level,id
8638,Першотравенська загальноосвітня школа I-III ст...,середня загальноосвітня школа,Кіровоградська область,Компаніївський район,с.Першотравенка,Відділ освіти Компаніївської районної державно...,Кіровоградська,Компаніївський,I-III,першотравенськазагальноосвтняшкола3ступенавчал...


finance


,region,teritname,schoolname,level of school,tername,id
6095,Кіровоградська,Компаніївський,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,Компаніївський,першотравенськазагальноосвтняшкола3ступенавчал...
6207,Кіровоградська,ОТГ Компаніївська,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,Компаніївський,першотравенськазагальноосвтняшкола3ступенавчал...


resolving duplicates

In [40]:
eie_schools_temp1.loc[8638, 'id'] = 'комп'+eie_schools_temp1.loc[8638, 'id']
finance_schools_temp1.loc[6095, 'id'] = 'комп'+finance_schools_temp1.loc[6095, 'id']

checking after removing

In [41]:
for region in sorted(eie_schools_temp1.region.unique()):
    region_schools = eie_schools_temp1[eie_schools_temp1.region == region]
    region_finance = finance_schools_temp1[finance_schools_temp1.region == region]
    for id in region_schools.id:
        if (id in region_finance.id.str[:len(id)].unique()) and \
             re.findall(r'№(\d+)', region_schools.loc[region_schools.id == id, 'eoname'].iloc[0]) == \
                re.findall(r'№(\d+)', region_finance.loc[region_finance.id.str[:len(id)] == id, 'schoolname'].iloc[0]):
            if len(region_schools[region_schools.id == id])>1 or len(region_finance[region_finance.id.str[:len(id)] == id]) >1:
                print('eie')
                display(region_schools[region_schools.id == id])
                print('finance')
                display(region_finance[region_finance.id.str[:len(id)] == id])

matching

In [42]:
for region in sorted(eie_schools_temp1.region.unique()):
    region_schools = eie_schools_temp1[eie_schools_temp1.region == region]
    region_finance = finance_schools_temp1[finance_schools_temp1.region == region]
    for id in region_schools.id:
        if (id in region_finance.id.str[:len(id)].unique()) and \
             re.findall(r'№(\d+)', region_schools.loc[region_schools.id == id, 'eoname'].iloc[0]) == \
                re.findall(r'№(\d+)', region_finance.loc[region_finance.id.str[:len(id)] == id, 'schoolname'].iloc[0]):
            df = pd.concat([df, pd.DataFrame([{'eoname': region_schools.loc[region_schools.id == id, 'eoname'].iloc[0], \
                                            'schoolname': region_finance.loc[region_finance.id.str[:len(id)] == id, 'schoolname'].iloc[0]}])], ignore_index=True)
            eie_schools_temp2 = eie_schools_temp2[~((eie_schools_temp2.region == region) & (eie_schools_temp2.id == id))]
            finance_schools_temp2 = finance_schools_temp2[~((finance_schools_temp2.region == region) & (finance_schools_temp2.id.str[:len(id)] == id))]

df.shape

(2991, 2)

In [43]:
df.tail(10)

,eoname,schoolname
2981,Агайманська загальноосвітня школа I-III ступенів,Агайманська загальноосвітня школа І-ІІІ ступен...
2982,Шарнопільський заклад загальної середньої осві...,Шарнопільський заклад загальної середньої осві...
2983,Бабанська загальноосвітня школа І-ІІІ ступенів...,Бабанська загальноосвітня школа І-ІІІ ступенів...
2984,"Опорний навчальний заклад ""Білозірська загальн...","Опорний навчальний заклад ""Білозірська загальн..."
2985,Ладижинська загальноосвітня школа І-ІІІ ступен...,Ладижинська загальноосвітня школа І-ІІІ ступен...
2986,"Шполянський навчально-виховний комплекс ""Загал...",Шполянський навчально-виховний комплекс «Загал...
2987,Дмитрушківська загальноосвітня школа І-ІІІ сту...,Дмитрушківська загальноосвітня школа І-ІІІ сту...
2988,"Городищенський навчально-виховний комплекс ""за...","Городищенський навчально-виховний комплекс ""за..."
2989,Батуринська загальноосвітня школа I-III ступен...,Батуринська загальноосвітня школа I-III ступен...
2990,"Гімназія ""Київська Русь""","гімназія ""Київська Русь"" Дарницького району м...."


In [44]:
print(eie_schools.shape[0]-eie_schools_temp2.shape[0])
print(finance_schools.shape[0] - finance_schools_temp2.shape[0])
print(df.shape[0])

2991
2991
2991


matching by trimming in different direction

In [45]:
eie_schools_temp3 = eie_schools_temp2
finance_schools_temp3 = finance_schools_temp2

In [46]:
eie_schools_temp2['level'] = eie_schools_temp2['level'].str.replace('І-ІІІ', 'I-III')
finance_schools_temp2['level of school'] = finance_schools_temp2['level of school'].str.replace('І-ІІІ', 'I-III')

In [47]:
eie_schools_temp2['level'] = eie_schools_temp2['level'].fillna('')
finance_schools_temp2['level of school'] = finance_schools_temp2['level of school'].fillna('')

In [48]:
eie_schools_temp2.loc[4587, 'eoname']

'Макарівський навчально-виховний комплекс "Загальноосвітня школа І ступеня - районна гімназія" Макарівської районної ради Київської області'

In [49]:
for region in sorted(eie_schools_temp2.region.unique()):
    region_schools = eie_schools_temp2[eie_schools_temp2.region == region]
    region_finance = finance_schools_temp2[finance_schools_temp2.region == region]
    for ternam in sorted(region_schools.tername.unique()):
        region_schools_ter = region_schools[region_schools.tername == ternam]
        region_finance_ter = region_finance[region_finance.tername == ternam]
        
        for lev in sorted(region_schools_ter.level.unique()):
            region_schools_ter_lev = region_schools_ter[region_schools_ter.level == lev]
            region_finance_ter_lev = region_finance_ter[region_finance_ter['level of school'] == lev]
            for id in region_finance_ter_lev.id:
                if (id in region_schools_ter_lev.id.str[:len(id)].unique()) and \
                    re.findall(r'№(\d+)', region_schools_ter_lev.loc[region_schools_ter_lev.id.str[:len(id)] == id, 'eoname'].iloc[0]) == \
                        re.findall(r'№(\d+)', region_finance_ter_lev.loc[region_finance_ter_lev.id == id, 'schoolname'].iloc[0]):
                    if len(region_schools_ter_lev[region_schools_ter_lev.id.str[:len(id)] == id])>1 or len(region_finance_ter_lev[region_finance_ter_lev.id == id]) >1:
                        print('data')
                        display(region_schools_ter_lev[region_schools_ter_lev.id.str[:len(id)] == id])
                        print('finance')
                        display(region_finance_ter_lev[region_finance_ter_lev.id == id])

data


,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,region,tername,level,id
3399,Рахнівсько-Лісова середня загальноосвітня школ...,середня загальноосвітня школа,Вінницька область,Шаргородський район,с.Рахни Лісові,Відділ освіти Шаргородської районної державної...,Вінницька,Шаргородський,I-III,рахнвськолсовасереднязагальноосвтняшкола3ступе...
8119,РАХНІВСЬКО-ЛІСОВИЙ НАВЧАЛЬНО-ВИХОВНИЙ КОМПЛЕКС...,навчально-виховний комплекс,Вінницька область,Шаргородський район,с.Рахни Лісові,Відділ освіти Шаргородської районної державної...,Вінницька,Шаргородський,I-III,рахнвськолсовийнавчальновиховнийкомплекс“серед...


finance


,region,teritname,schoolname,level of school,tername,id
736,Вінницька,Шаргородський,Рахнівсько-Ліс,I-III,Шаргородський,рахнвськолс


data


...

,region,teritname,schoolname,level of school,tername,id
12473,Хмельницька,Кам-Подільський район,Орининська,I-III,Кам'янець-Подільський,орининська


data


,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,region,tername,level,id
1923,Школа І-ІІІ ступенів № 259 Деснянського району...,середня загальноосвітня школа,м.Київ,м.Київ. Деснянський район міста,Деснянський район міста,Управління освіти Деснянської районної в м. Ки...,місто Київ,м.Київ,I-III,школа3ступенв№259деснянськогорайонумкиєва
3182,Школа І-ІІІ ступенів №256 Оболонського району ...,середня загальноосвітня школа,м.Київ,м.Київ. Оболонський район міста,Оболонський район міста,Управління освіти Оболонської районної в м. Ки...,місто Київ,м.Київ,I-III,школа3ступенв№256оболонськогорайонумкиєва


finance


,region,teritname,schoolname,level of school,tername,id
14739,місто Київ,Київ- Шевченківський р-н,Школа І-ІІІ ступенів № 25,I-III,м.Київ,школа3ступенв№25


In [50]:
eie_schools_temp2.loc[3399, 'id'] = 'шар'+eie_schools_temp2.loc[3399, 'id']
finance_schools_temp2.loc[736, 'id'] = 'шар'+finance_schools_temp2.loc[736, 'id']

In [51]:
eie_schools_temp2.loc[3519, 'id'] = 'раф'+eie_schools_temp2.loc[3519, 'id']
finance_schools_temp2.loc[9580, 'id'] = 'раф'+finance_schools_temp2.loc[9580, 'id']

In [52]:
eie_schools_temp2.loc[7084, 'id'] = 'ром'+eie_schools_temp2.loc[7084, 'id']
finance_schools_temp2.loc[3124, 'id'] = 'ром'+finance_schools_temp2.loc[3124, 'id']

In [53]:
eie_schools_temp2.loc[8373, 'id'] = 'ор'+eie_schools_temp2.loc[8373, 'id']
finance_schools_temp2.loc[12473, 'id'] = 'ор'+finance_schools_temp2.loc[12473, 'id']

In [54]:
finance_schools_temp2.loc[14739, 'id'] = finance_schools_temp2.loc[14739, 'id']+'_'

In [55]:
for region in sorted(eie_schools_temp2.region.unique()):
    region_schools = eie_schools_temp2[eie_schools_temp2.region == region]
    region_finance = finance_schools_temp2[finance_schools_temp2.region == region]
    for ternam in sorted(region_schools.tername.unique()):
        region_schools_ter = region_schools[region_schools.tername == ternam]
        region_finance_ter = region_finance[region_finance.tername == ternam]
        
        for lev in sorted(region_schools_ter.level.unique()):
            region_schools_ter_lev = region_schools_ter[region_schools_ter.level == lev]
            region_finance_ter_lev = region_finance_ter[region_finance_ter['level of school'] == lev]
            for id in region_finance_ter_lev.id:
                if (id in region_schools_ter_lev.id.str[:len(id)].unique()) and \
                    re.findall(r'№(\d+)', region_schools_ter_lev.loc[region_schools_ter_lev.id.str[:len(id)] == id, 'eoname'].iloc[0]) == \
                        re.findall(r'№(\d+)', region_finance_ter_lev.loc[region_finance_ter_lev.id == id, 'schoolname'].iloc[0]):
                    if len(region_schools_ter_lev[region_schools_ter_lev.id.str[:len(id)] == id])>1 or len(region_finance_ter_lev[region_finance_ter_lev.id == id]) >1:
                        print('data')
                        display(region_schools_ter_lev[region_schools_ter_lev.id.str[:len(id)] == id])
                        print('finance')
                        display(region_finance_ter_lev[region_finance_ter_lev.id == id])

In [56]:
for region in sorted(eie_schools_temp2.region.unique()):
    region_schools = eie_schools_temp2[eie_schools_temp2.region == region]
    region_finance = finance_schools_temp2[finance_schools_temp2.region == region]
    for ternam in sorted(region_schools.tername.unique()):
        region_schools_ter = region_schools[region_schools.tername == ternam]
        region_finance_ter = region_finance[region_finance.tername == ternam]
        
        for lev in sorted(region_schools_ter.level.unique()):
            region_schools_ter_lev = region_schools_ter[region_schools_ter.level == lev]
            region_finance_ter_lev = region_finance_ter[region_finance_ter['level of school'] == lev]
            for id in region_finance_ter_lev.id:
                if (id in region_schools_ter_lev.id.str[:len(id)].unique()) and \
                    re.findall(r'№(\d+)', region_schools_ter_lev.loc[region_schools_ter_lev.id.str[:len(id)] == id, 'eoname'].iloc[0]) == \
                        re.findall(r'№(\d+)', region_finance_ter_lev.loc[region_finance_ter_lev.id == id, 'schoolname'].iloc[0]):
                    df = pd.concat([df, pd.DataFrame([{'eoname':region_schools_ter_lev.loc[region_schools_ter_lev.id.str[:len(id)] == id, 'eoname'].iloc[0], \
                                            'schoolname': region_finance_ter_lev.loc[region_finance_ter_lev.id == id, 'schoolname'].iloc[0]}])], ignore_index=True)
                    eie_schools_temp3 = eie_schools_temp3[~((eie_schools_temp3.region == region) & (eie_schools_temp3.tername == ternam)& (eie_schools_temp3.level == lev) &(eie_schools_temp3.id.str[:len(id)] == id))]
                    finance_schools_temp3 = finance_schools_temp3[~((finance_schools_temp3.region == region) & (finance_schools_temp3.tername == ternam)& (finance_schools_temp3['level of school'] == lev) &(finance_schools_temp3.id == id))]

df.shape

(4484, 2)

In [57]:
print(eie_schools.shape[0]-eie_schools_temp3.shape[0])
print(finance_schools.shape[0] - finance_schools_temp3.shape[0])
print(df.shape[0])

4484
4484
4484


In [58]:
df.tail(10)

,eoname,schoolname
4474,Спеціалізована школа І-ІІІ ступенів №53 з погл...,Спеціалізована школа І-ІІІ ступенів №53 з погл...
4475,Спеціалізована школа І-ІІІ ступенів №57 з погл...,Спеціалізована школа І-ІІІ ступенів №57 з погл...
4476,Спеціалізована школа І-ІІІ ступенів №61 з погл...,Спеціалізована школа І-ІІІ ступенів №61 з погл...
4477,Спеціалізована школа І-ІІІ ступенів №73 з погл...,Спеціалізована школа І-ІІІ ступенів №73 з погл...
4478,Спеціалізована школа І-ІІІ ступенів №91 з погл...,Спеціалізована школа І-ІІІ ступенів №91 з погл...
4479,Спеціалізована школа І-ІІІ ступенів №102 з пог...,Спеціалізована школа І-ІІІ ступенів №102 з пог...
4480,Спеціалізована школа І-ІІІ ступенів №135 з пог...,Спеціалізована школа І-ІІІ ступенів №135 з пог...
4481,Спеціалізована школа І-ІІІ ступенів №138 з пог...,Спеціалізована школа І-ІІІ ступенів №138 з пог...
4482,Спеціалізована школа І-ІІІ ступенів №139 з пог...,Спеціалізована школа І-ІІІ ступенів №139 з пог...
4483,Спеціалізована школа І-ІІІ ступенів №155 з пог...,Спеціалізована школа І-ІІІ ступенів №155 з пог...


downloading dataset that has information about the status (private or public) iof schools

In [59]:
schools = pd.read_excel('Заклади загальної середньої освіти.xlsx')
schools

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,Населений пункт,...,Телефон,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги
0,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,працює,гімназія,Комунальна,8036100000,КИЇВ,Київ,...,(044)2584242,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,1800.0
1,гімназія №59 імені О.М. Бойченка міста Києва,148414,так,гімназія №59,працює,гімназія,Комунальна,8036100000,КИЇВ,Київ,...,(044)5252138,(044)5252138,gymnasium59@ukr.net,http://gymnasium59.org.ua/,Директор Антоненко Тетяна Стефанівна,ні,ні,ні,ні,1800.0
2,Еколого-природничий ліцей №116 міста Києва,148429,так,Еколого-природничий ліцей №116 міста Києва,працює,ліцей,Комунальна,8036100000,КИЇВ,Київ,...,(044)2589988,(044)2589988,ecoprirodnichi116@gmail.com,http://eco116.at.ua/,Директор Авраменко Таміла Олексіївна,ні,ні,ні,ні,NaN
3,"Заклад освіти ""Приватний загальноосвітній навч...",148411,так,"ЗО ""ПЗНЗ ""МІДГАРД""",працює,школа,Приватна,8036100000,КИЇВ,Київ,...,(098)888-09-99,NaN,midgard.kiev@gmail.com,http://www.midgard.school,Директор Попов Олег Анатолійович,ні,ні,ні,ні,600.0
4,Ліцей «Гранд»,148412,так,Ліцей «Гранд»,працює,ліцей,Приватна,8036100000,КИЇВ,Київ,...,(044)2871457,NaN,school@grand.kiev.ua,www.grandschool.com.ua,Директор Зайчук Світлана Анатоліївна,ні,ні,ні,ні,250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16254,"комунальний заклад ""Яблунівський ліцей"" Черніг...",139767,так,"КЗ ""Яблунівський ліцей""",працює,ліцей,Комунальна,7424189701,Чернігівська область,"с. Яблунівка, Прилуцький район, Чернігівська о...",...,"(04637)65238, (04637)65282",NaN,Yablunovka@ukr.net,NaN,Директор Шаповаленко Неоніла Олександрівна,Опорний заклад,так,ні,так,170.0
16255,Яблунівська загальноосвітня школа I-III ступен...,144175,так,Яблунівська ЗОШ І-ІІІ ступенів,працює,школа,Комунальна,7424189701,Чернігівська область,"с. Яблунівка, Прилуцький район, Чернігівська о...",...,(04637)6-52-42,NaN,yablunivkask@ukr.net,NaN,Директор Колесник Анатолій Васильович,ні,так,ні,ні,320.0
16256,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,працює,школа,Комунальна,7420889601,Чернігівська область,"с. Ядути, Борзнянський район, Чернігівська обл...",...,(04653)27571,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1.0
16257,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,працює,школа,Комунальна,7424781701,Чернігівська область,"с. Янжулівка, Семенівський район, Чернігівська...",...,(046)5924644,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90.0


In [60]:
schools.shape

(16259, 23)

In [61]:
schools_private = schools[schools['Форма власності'] == 'Приватна']

In [62]:
schools_private['id'] = schools_private['Повна назва']
abbr = {'КЗ ':'Комунальний заклад ', 'ОНЗ ':'Опорний навчальний заклад ', 'ОЗЗСО ':'Опорний заклад загальної середньої освіти ',
        'ЗЗСО ':'Заклад загальної середньої освіти ', 'ООЗ ':'Опорний освітній заклад ', 'ЗСО ':'Загальної середньої освіти ', 
        'НВК ':'Навчально-виховний комплекс ', 'ЗОШ ':'Загальнооосвітня школа ', 'СЗШ ':'Середня загальноосвітня школа ',
        'ДНЗ ':'Дошкільний навчальний заклад '}

for ab in abbr:
    schools_private['id'] = schools_private['id'].str.replace(ab, abbr[ab])

schools_private['id'] = schools_private['id'].str.replace('[\s-]', '', regex = True)
schools_private.loc[schools_private['id'].str.endswith('обл'), 'id'] = schools_private.loc[schools_private['id'].str.endswith('обл'), 'id'] + '.'
schools_private['id'] = schools_private['id'].str.replace('І', 'I')
schools_private['id'] = schools_private['id'].str.replace('I-III', '3')
schools_private['id'] = schools_private['id'].str.replace('I-II', '2')
schools_private['id'] = schools_private['id'].str.lower()

dct = {'с.':'села', 'ст.':'ступенів', 'ім.':'імені', 'iм.':'імені', 'обл.': 'області', 
       '||||':'3', '|||':'2',
       'i':'', 'і':'', 'ї':'', 'o':'о', 'y':'у', 'p':'р', 'c':'с', 'ґ':'', 
       "'":'', "’":'', ",":'', ".":'', "«":'', "»":'', "–":'', "-":'', "*":"",
       "_x000d_":'', '_x000D_':'', "_":'', "(":'', ")":'', "[":'', "]":'', "}":'', "{":'', 
       "нвк":'навчальновиховнийкомплекс', 'знз': 'загальноосвтнйнавчальнийзаклад', 'зош': 'загальноосвтняшкола', 
       'сзш':'середнязагальноосвтняшкола', 'нн':'н', 'днз': 'дошкльнийнавчальнийзаклад'}

for symb in dct:
    schools_private['id'] = schools_private['id'].str.replace(symb, dct[symb])

schools_private['id'] = schools_private['id'].str.replace('"', '', regex = True)


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_21120/2356282020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_private['id'] = schools_private['Повна назва']
/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_21120/2356282020.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_private['id'] = schools_private['id'].str.replace(ab, abbr[ab])
/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_21120/2356282020.py:10: SettingWithCopyWarning: 
A value is trying to 

In [63]:
schools_private

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,Населений пункт,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
3,"Заклад освіти ""Приватний загальноосвітній навч...",148411,так,"ЗО ""ПЗНЗ ""МІДГАРД""",працює,школа,Приватна,8036100000,КИЇВ,Київ,...,NaN,midgard.kiev@gmail.com,http://www.midgard.school,Директор Попов Олег Анатолійович,ні,ні,ні,ні,600.0,закладосвтиприватнийзагальноосвтнйнавчальнийза...
4,Ліцей «Гранд»,148412,так,Ліцей «Гранд»,працює,ліцей,Приватна,8036100000,КИЇВ,Київ,...,NaN,school@grand.kiev.ua,www.grandschool.com.ua,Директор Зайчук Світлана Анатоліївна,ні,ні,ні,ні,250.0,лцейгранд
10,МІЖНАРОДНИЙ ХРИСТИЯНСЬКИЙ ПРИВАТНИЙ ЗАКЛАД ЗАГ...,176484,так,МХЛ м. Київ «ДІТИ ЦАРЯ»,працює,ліцей,Приватна,8036100000,КИЇВ,Київ,...,NaN,kingkids@ukr.net,https://kingskids-school.com/,Директор Богорад Алла Олександрівна,ні,ні,ні,ні,NaN,мжнароднийхристиянськийприватнийзакладзагально...
14,"Навчально-виховний комплекс ""Міжнародний ліцей...",144362,так,"НВК ""Міжнародний ліцей МАУП""",працює,навчально-виховний комплекс (об'єднання),Приватна,8036100000,КИЇВ,Київ,...,NaN,liceum@ilmaup.com.ua,ilmaup.com.ua,Директор Зеленська Зоя Петрівна,ні,ні,ні,ні,NaN,навчальновиховнийкомплексмжнароднийлцеймжрегон...
15,"освітній заклад ""ШКОЛА ""ГРАНД-ЕКСПО"" (структур...",176486,так,"ПП ""МОК ""ГРАНД-ЕКСПО""",працює,школа,Приватна,8036100000,КИЇВ,Київ,...,NaN,NaN,NaN,Директор Веприцька Тетяна Володимирівна,ні,ні,ні,ні,NaN,освтнйзакладшколаграндекспоструктурнийпдроздлп...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15374,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ЧЕРНІ...",176567,так,"ТОВ ""ЧЕРНІВЕЦЬКИЙ ПРИВАТНИЙ ЛІЦЕЙ ""ЮНІК""",працює,ліцей,Приватна,7310100000,Чернівецька область,"Чернівці, Чернівецька область",...,NaN,NaN,NaN,Директор Куницька Ірина Михайлівна,ні,ні,ні,ні,NaN,товариствозобмеженоювдповдальнстючернвецькийпр...
15801,"ПРИВАТНЕ ПІДПРИЄМСТВО ""ПРИВАТНИЙ ЗАКЛАД ОСВІТИ...",148340,так,"ПП ""ПЗО ""МУДРАНГЕЛИКИ""",працює,школа,Приватна,7410136300,Чернігівська область,"Чернігів, Чернігівська область",...,NaN,mudrangelschool@gmail.com,https://mudrangels.com/,Директор Лавріненко Неля Петрівна,ні,ні,ні,ні,NaN,приватнепдприємствоприватнийзакладосвтимудранг...
15804,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ЛІЦЕЙ...",176538,так,"ТОВ ""ЛІЦЕЙ ""РОБОСКУЛ""",працює,ліцей,Приватна,7410100000,Чернігівська область,"Чернігів, Чернігівська область",...,NaN,info@roboschool.org.ua,NaN,Директор Лимар Сергій Миколайович,ні,ні,ні,ні,NaN,товариствозобмеженоювдповдальнстюлцейробоскулм...
15805,"Товариство з обмеженою відповідальністю ""Прива...",176496,так,"ТОВ ""Приватна дистанційна загальноосвітня школ...",працює,школа,Приватна,7410100000,Чернігівська область,"Чернігів, Чернігівська область",...,NaN,manager.redfox@gmail.com,NaN,Директор Василенко Неля Анатоліївна,ні,ні,ні,ні,300.0,товариствозобмеженоювдповдальнстюприватнадиста...


In [64]:
eie_schools_temp4 = eie_schools_temp3

removing from our dataset for matching all private schools

In [65]:
for id in eie_schools_temp3.id:
    if id in schools_private.id.str[:len(id)].unique():
        df = pd.concat([df, pd.DataFrame([{'eoname': eie_schools_temp3.loc[eie_schools_temp3['id'] == id, 'eoname'].iloc[0], \
                                           'schoolname': 'private'}])], ignore_index=True)
        eie_schools_temp4 = eie_schools_temp4[eie_schools_temp4.id != id]

df.shape

(4523, 2)

In [66]:
print(eie_schools.shape[0]-eie_schools_temp4.shape[0])
print(finance_schools.shape[0] - finance_schools_temp3.shape[0]+39)
print(df.shape[0])

4523
4523
4523


In [67]:
eie_schools_temp5 = eie_schools_temp4
finance_schools_temp5 = finance_schools_temp3

matching by contains

In [68]:
for region in sorted(eie_schools_temp4.region.unique()):
    region_schools = eie_schools_temp4[eie_schools_temp4.region == region]
    region_finance = finance_schools_temp3[finance_schools_temp3.region == region]
    for id in region_schools.id:
        matching_long_values = region_finance[region_finance.id.str.contains(id)]
        if not matching_long_values.empty and \
            re.findall(r'№(\d+)', region_schools.loc[region_schools.id == id, 'eoname'].iloc[0]) == \
                re.findall(r'№(\d+)', region_finance.loc[region_finance.id.str.contains(id), 'schoolname'].iloc[0]):
            
            if len(matching_long_values)>1 or len(region_schools[region_schools.id == id])>1:
                print('data')
                display(region_schools[region_schools.id == id])
                print('finance')
                display(matching_long_values)

data


,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,region,tername,level,id
7153,Заклад загальної середньої освіти І-ІІІ ступен...,середня загальноосвітня школа,Донецька область,м.Торецьк,м.Торецьк,Міське управління освіти військово-цивільної а...,Донецька,м.Торецьк,I-III,закладзагальносередньоосвти3ступенв№2


finance


,region,teritname,schoolname,level of school,tername,id
2399,Донецька,м.Добропілля,Добропільський заклад загальної середньої осві...,I-III,м.Добропілля,добропльськийзакладзагальносередньоосвти3ступе...
2837,Донецька,"Сіверська ОТГ, всього, в тому числі:",Сіверський заклад загальної середньої освіти І...,I-III,Сіверський,сверськийзакладзагальносередньоосвти3ступенв№2...


In [69]:
eie_schools_temp4.loc[7153, 'id'] = 'торецьк'+eie_schools_temp4.loc[7153, 'id']

In [70]:
for region in sorted(eie_schools_temp4.region.unique()):
    region_schools = eie_schools_temp4[eie_schools_temp4.region == region]
    region_finance = finance_schools_temp3[finance_schools_temp3.region == region]
    for id in region_schools.id:
        matching_long_values = region_finance[region_finance.id.str.contains(id)]
        if not matching_long_values.empty and \
            re.findall(r'№(\d+)', region_schools.loc[region_schools.id == id, 'eoname'].iloc[0]) == \
                re.findall(r'№(\d+)', region_finance.loc[region_finance.id.str.contains(id), 'schoolname'].iloc[0]):

            df = pd.concat([df, pd.DataFrame([{'eoname': region_schools.loc[region_schools.id == id, 'eoname'].iloc[0], \
                                            'schoolname': region_finance.loc[region_finance['id'].str.contains(id), 'schoolname'].iloc[0]}])], ignore_index=True)
            eie_schools_temp5 = eie_schools_temp5[~((eie_schools_temp5.region == region) & (eie_schools_temp5.id == id))]
            finance_schools_temp5 = finance_schools_temp5[~((finance_schools_temp5.region == region) & (finance_schools_temp5.id.str.contains(id)))]

df.shape

(4557, 2)

In [71]:
df.tail()

,eoname,schoolname
4552,Борівська загальноосвітня школа I-III ступенів...,Комунальний заклад «Борівська загальноосвітня ...
4553,Гімназія Корсунь-Шевченківської районної ради ...,Корсунь-Шевченківська гімназія Корсунь-Шевчен...
4554,"Боярський навчально-виховний комплекс ""Дошкіль...","Комунальний опорний заклад ""Боярський навчальн..."
4555,"Деснянський навчально-виховний комплекс ""Гімна...","Опорний заклад ""Деснянський навчально-виховний..."
4556,Гімназія міжнародних відносин №323 з поглиблен...,загальноосвітній навчальний заклад І-ІІІ ступе...


In [72]:
print(eie_schools.shape[0]-eie_schools_temp5.shape[0])
print(finance_schools.shape[0] - finance_schools_temp5.shape[0]+39)
print(df.shape[0])

4557
4557
4557


In [73]:
eie_schools_temp6 = eie_schools_temp5
finance_schools_temp6 = finance_schools_temp5

In [74]:
for region in sorted(eie_schools_temp5.region.unique()):
    region_schools = eie_schools_temp5[eie_schools_temp5.region == region]
    region_finance = finance_schools_temp5[finance_schools_temp5.region == region]
    for ternam in sorted(region_schools.tername.unique()):
        region_schools_ter = region_schools[region_schools.tername == ternam]
        region_finance_ter = region_finance[region_finance.tername == ternam]

        for lev in sorted(region_schools_ter.level.unique()):
            region_schools_ter_lev = region_schools_ter[region_schools_ter.level == lev]
            region_finance_ter_lev = region_finance_ter[region_finance_ter['level of school'] == lev]
            for id in region_finance_ter_lev.id:
                matching_long_values = region_schools_ter_lev[region_schools_ter_lev.id.str.contains(id)]
                if not matching_long_values.empty and \
                    re.findall(r'№(\d+)', region_schools_ter_lev.loc[region_schools_ter_lev.id.str.contains(id), 'eoname'].iloc[0]) == \
                        re.findall(r'№(\d+)', region_finance_ter_lev.loc[region_finance_ter_lev.id == id, 'schoolname'].iloc[0]):
                    
                    if len(matching_long_values)>1:
                        print(id)
                        print('data')
                        display(matching_long_values)
                        print('finance')
                        display(region_finance_ter_lev[region_finance_ter_lev.id == id])

In [75]:
for region in sorted(eie_schools_temp5.region.unique()):
    region_schools = eie_schools_temp5[eie_schools_temp5.region == region]
    region_finance = finance_schools_temp5[finance_schools_temp5.region == region]
    for ternam in sorted(region_schools.tername.unique()):
        region_schools_ter = region_schools[region_schools.tername == ternam]
        region_finance_ter = region_finance[region_finance.tername == ternam]

        for lev in sorted(region_schools_ter.level.unique()):
            region_schools_ter_lev = region_schools_ter[region_schools_ter.level == lev]
            region_finance_ter_lev = region_finance_ter[region_finance_ter['level of school'] == lev]
            for id in region_finance_ter_lev.id:
                matching_long_values = region_schools_ter_lev[region_schools_ter_lev.id.str.contains(id)]
                if not matching_long_values.empty and \
                    re.findall(r'№(\d+)', region_schools_ter_lev.loc[region_schools_ter_lev.id.str.contains(id), 'eoname'].iloc[0]) == \
                        re.findall(r'№(\d+)', region_finance_ter_lev.loc[region_finance_ter_lev.id == id, 'schoolname'].iloc[0]):
                    
                    df = pd.concat([df, pd.DataFrame([{'eoname': region_schools_ter_lev.loc[region_schools_ter_lev.id.str.contains(id), 'eoname'].iloc[0], \
                                                    'schoolname': region_finance_ter_lev.loc[region_finance_ter_lev.id == id, 'schoolname'].iloc[0]}])], ignore_index=True)
                    eie_schools_temp6 = eie_schools_temp6[~((eie_schools_temp6.region == region) & (eie_schools_temp6.tername == ternam)& (eie_schools_temp6.level == lev)&(eie_schools_temp6.id.str.contains(id)))]
                    finance_schools_temp6 = finance_schools_temp6[~((finance_schools_temp6.region == region) & (finance_schools_temp6.tername == ternam)& (finance_schools_temp6['level of school'] == lev) & (finance_schools_temp6.id == id))]


df.shape

(4596, 2)

In [76]:
df.tail()

,eoname,schoolname
4591,Загальноосвітній навчальний заклад І-ІІІ ступе...,«Романо-германська гімназія № 123»
4592,Загальноосвітній навчальний заклад І-ІІІ ступе...,«Середня загальноосвітня школа № 156»
4593,Загальноосвітній навчальний заклад І-ІІІ ступе...,«Середня загальноосвітня школа № 242»
4594,Загальноосвітній навчальний заклад І-ІІІ ступе...,«Середня загальноосвітня школа № 243»
4595,Загальноосвітній навчальний заклад І-ІІІ ступе...,«Середня загальноосвітня школа № 262»


In [77]:
print(eie_schools.shape[0]-eie_schools_temp6.shape[0])
print(finance_schools.shape[0] - finance_schools_temp6.shape[0]+39)
print(df.shape[0])

4596
4596
4596


In [78]:
eie_schools_temp6.to_csv('zno_schools_unmatched_2018.csv', index=False)

In [79]:
finance_schools_temp6.to_csv('finance_schools_unmatched_2018.csv', index=False)

In [80]:
df.to_csv('finance_schools_matched_2018.csv', index=False)

In [81]:
print('need to match', eie_schools.shape[0]-df.shape[0], 'schools')

need to match 5177 schools


In [82]:
print('matched', round(df.shape[0]/eie_schools.shape[0]*100), '% of schools')

matched 47 % of schools
